In [ ]:
![ -d QAOAKit ] || git clone https://github.com/melody0123/QAOAKit.git

import os
os.chdir("QAOAKit")
print(os.getcwd()) # debugging output

!pip install .

os.chdir("..")
print(os.getcwd()) # debugging output

In [ ]:
!pip install pytket
!pip install pyzx
!pip install 'trasyn[qiskit,cupy-cuda12]'
!pip install matplotlib pylatexenc

In [ ]:
!python -m QAOAKit.build_tables

In [ ]:
import networkx as nx
from qiskit_aer import AerSimulator
from QAOAKit import opt_angles_for_graph, angles_to_qaoa_format
from QAOAKit.qaoa import get_maxcut_qaoa_circuit

# build a random 3-regular graph with 10 nodes
graph = nx.random_regular_graph(d=3, n=10, seed=314)

# grab optimal angles
p = 3
angles = angles_to_qaoa_format(opt_angles_for_graph(graph,p))

# build circuit
qc = get_maxcut_qaoa_circuit(graph, angles['beta'], angles['gamma'])
qc.measure_all()

# run circuit
backend = AerSimulator()
print(backend.run(qc).result().get_counts())

In [ ]:
# remove save_state operation
new_qc = qc.copy_empty_like()
for inst in qc:
    op = inst.operation
    if op.name != 'save_state':
        new_qc.append(op, inst.qubits, inst.clbits)

# new_qc.draw(output='mpl')

In [ ]:
import trasyn

syned_qc = trasyn.synthesize_qiskit_circuit(circuit=new_qc, u3_transpile=True, nonclifford_budget=50)